In [1]:
#Get the dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [1]:
#Get Input Text
with open('input.txt', 'r') as f:
    text = f.read()

In [2]:
#Get All Unique Characters
chars = sorted(list(set(text)))
print(chars)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [3]:
#Tokenize the text- This will be used to convert the text to numbers
char_to_idx = {ch:i for i,ch in enumerate(chars)}
idx_to_char = {i:ch for i,ch in enumerate(chars)}

#Encode and Decode Functions for Strings
def encode(s):
    return [char_to_idx[ch] for ch in s]
def decode(l):
    return ''.join([idx_to_char[i] for i in l])

#Test
encodedText = encode("Hello!")
print(encodedText)
print(decode(encodedText))

#Note: This is a very simple tokenization method. There are better ways to do this. Example: Using tiktoken from OpenAI. (Sub-word tokenization = Don't need a new token for every letter)


[20, 43, 50, 50, 53, 2]
Hello!


In [4]:
#Some Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

In [6]:
#Encode the input text and store it into a Torch Tensor
#A tensor is a multi-dimensional matrix containing elements of a single data type

#Convert the text to numbers
encodedText = encode(text)
print("Length of encoded text: ", len(encodedText))

#Convert the list to a tensor
data = torch.tensor(encodedText, dtype=torch.long)
print(data.shape)
print(data[:10]) #First 10 elements (First 10 characters in the text)


Length of encoded text:  1115394
torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [7]:
#Get Training and Validation Data
#We will use the first 90% of the data for training and the last 10% for validation

#Get the length of the data
dataLen = len(data)
print("Length of data: ", dataLen)

#Get the length of the training data  
trainLen = int(dataLen * 0.9)

#Get the training data
train_data = data[:trainLen]
print("Length of training data: ", len(train_data))

#Get the validation data
val_data = data[trainLen:]  
print("Length of validation data: ", len(val_data))

Length of data:  1115394
Length of training data:  1003854
Length of validation data:  111540


In [8]:
block_size = 8 #The length of the sequence we want to train on (the context)
#Example: If block_size = 8, then we want to predict the 9th character in the sequence
#This is why we will need to train the model on sequences of length block_size + 1
#In the below code the max value of t+1 is block_size

x = train_data[0:block_size] # The input sequence
y = train_data[1:block_size+1] # The output sequence

print("Input sequence: ", x)
print("Output sequence: ", y)

for t in range(block_size): 
    context = x[:t+1]
    target = y[t]
    print(t+1, ': Context: ', str(context), ' Target: ', str(target)) 
    #Ex. If data is {1,2,3} and we are given 1, then the target (predicted output) should be 2 

Input sequence:  tensor([18, 47, 56, 57, 58,  1, 15, 47])
Output sequence:  tensor([47, 56, 57, 58,  1, 15, 47, 58])
1 : Context:  tensor([18])  Target:  tensor(47)
2 : Context:  tensor([18, 47])  Target:  tensor(56)
3 : Context:  tensor([18, 47, 56])  Target:  tensor(57)
4 : Context:  tensor([18, 47, 56, 57])  Target:  tensor(58)
5 : Context:  tensor([18, 47, 56, 57, 58])  Target:  tensor(1)
6 : Context:  tensor([18, 47, 56, 57, 58,  1])  Target:  tensor(15)
7 : Context:  tensor([18, 47, 56, 57, 58,  1, 15])  Target:  tensor(47)
8 : Context:  tensor([18, 47, 56, 57, 58,  1, 15, 47])  Target:  tensor(58)


In [9]:
torch.manual_seed(1337) #Set the seed for reproducibility
block_size = 8 #The length of the sequence we want to train on 
batch_size = 4 #The number of sequences we want to train on at a time

def get_batch(split):
    #Get the data depending on the split (train or validation)
    if split == 'train':
        data = train_data
    else:
        data = val_data
    
    #Get batch_size random starting indexes
    ix = torch.randint(high=len(data)-block_size, size=(batch_size,))

    #Construct the input and output sequences
    x = [data[i:i+block_size] for i in ix]
    y = [data[i+1:i+block_size+1] for i in ix]
    
    #Convert the sequences to tensors of size (batch_size, block_size)
    x = torch.stack(x)
    y = torch.stack(y)

    return x, y

#Test
xb, yb = get_batch('train')
print("Input batch shape: ", xb.shape)
print("Output batch shape: ", yb.shape)

print("-----------------")
print("Input batch: ")
print(xb)
print("-----------------")
print("Output batch: ")
print(yb)
print("-----------------")

for t in range(block_size): #Basically the same as the previous but now we are using batches (multiple sequences)
    context = xb[:, :t+1] #Get the first t+1 elements of each sequence
    target = yb[:, t]
    print(t+1, ': Context: ', str(context), ' \nTarget: ', str(target)) 
    #Ex. If data is {1,2,3} and we are given 1, then the target (predicted output) should be 2

Input batch shape:  torch.Size([4, 8])
Output batch shape:  torch.Size([4, 8])
-----------------
Input batch: 
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
-----------------
Output batch: 
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----------------
1 : Context:  tensor([[24],
        [44],
        [52],
        [25]])  
Target:  tensor([43, 53, 58, 17])
2 : Context:  tensor([[24, 43],
        [44, 53],
        [52, 58],
        [25, 17]])  
Target:  tensor([58, 56,  1, 27])
3 : Context:  tensor([[24, 43, 58],
        [44, 53, 56],
        [52, 58,  1],
        [25, 17, 27]])  
Target:  tensor([ 5,  1, 58, 10])
4 : Context:  tensor([[24, 43, 58,  5],
        [44, 53, 56,  1],
        [52, 58,  1, 58],
        [25, 17, 27, 10]])  
Target:  t

In [68]:
torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        #super() returns the parent object of the class
        #In this case, it returns the parent object of BigramLanguageModel which is nn.Module (defined above)
        super().__init__() # Call the parent class constructor

        
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #Embedding layer
        #Embedding layer is used to convert the input tokens to vectors of size embedding_dim
        #In this case, the input tokens are numbers (the encoded text) and the embedding_dim is vocab_size
        #Vectors will allow us to do math operations on the tokens

    def forward(self, idx, targets=None):

        logits = self.token_embedding_table(idx)*.1 #Get the embeddings for the input tokens. (Batch, Time, Channel) or (Batch, Block_size, vocab_size)

        if targets != None:
            logits = logits.view(-1, logits.shape[-1]) 
            print(logits.shape)
            loss = F.cross_entropy(logits, targets.view(-1)) #Calculate the loss. Logits and targets need to be reshaped to (Batch * Block_size, vocab_size) and (Batch * Block_size) respectively
        else:
            loss = None
        


        return logits,loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits,loss = self(idx[:,-1])

            # logits,loss = self(idx)
            # probs = F.softmax((logits[:, -1]), dim=-1)
            probs = F.softmax((logits), dim=-1)
            new_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, new_token), dim=-1)
        return idx

m = BigramLanguageModel(len(chars)) #Create the model
out,loss = m(xb, yb) #Get the output

#Test Generate
idx = torch.zeros((1,1), dtype=torch.long)
out = ((m.generate(idx, 100).squeeze().tolist()))
print(decode(out))





torch.Size([32, 65])

bL?DP-QWkrEoL?jLDJQcLVf'RIHD'Hdhs Yv,ixa,jswYZwLEuS'paokqOzs$!A$zFGQT;eMk x.gQ$FCLg!iWn.O!zDGyA YsT3


In [52]:
#Create a pytorch optimizer
optimizer = torch.optim.Adam(m.parameters(), lr=0.0001)

In [53]:
batch_size = 32
for steps in range(10000): 
    #Get the input an0d output sequences
    xb, yb = get_batch('train')
    
    #Get the logits and loss
    logits, loss = m(xb, yb)
    
    #Zero the gradients
    optimizer.zero_grad()
    
    #Backpropagate the loss
    loss.backward()
    
    #Update the parameters
    optimizer.step()
    
    #Print the loss every 10 steps
    if steps % 1000 == 0:
        print("Step: ", steps, ", Loss: ", loss.item())

Step:  0 , Loss:  2.6501591205596924
Step:  1000 , Loss:  2.6225929260253906
Step:  2000 , Loss:  2.5740420818328857
Step:  3000 , Loss:  2.6518807411193848
Step:  4000 , Loss:  2.53884220123291
Step:  5000 , Loss:  2.667741298675537
Step:  6000 , Loss:  2.56286358833313
Step:  7000 , Loss:  2.527754306793213
Step:  8000 , Loss:  2.60963773727417
Step:  9000 , Loss:  2.5388073921203613


In [54]:
#Test Generate
idx = torch.zeros((1,1), dtype=torch.long)
out = ((m.generate(idx, 100).squeeze().tolist()))
print(decode(out))


Owhowee:

 chatwaishovzKBobrem,Pr
ymqPamoure:haisoue cear ve
vo llillil'Tblmead mpYvJall,ony btlwd w


In [64]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_emb = 32
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [71]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_embedding_table = nn.Embedding(block_size, n_emb)
        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):

        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        logits = self.lm_head(x) # (B,T, vocab_size)


        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [66]:
torch.cuda.is_available()

True

In [68]:
model = BigramLanguageModel()
m = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [70]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 2.4908, val loss 2.5071
step 300: train loss 2.4943, val loss 2.5115
step 600: train loss 2.4804, val loss 2.5053
step 900: train loss 2.4814, val loss 2.5106
step 1200: train loss 2.4815, val loss 2.5262
step 1500: train loss 2.4787, val loss 2.5169
step 1800: train loss 2.4758, val loss 2.5021
step 2100: train loss 2.4788, val loss 2.5090
step 2400: train loss 2.4903, val loss 2.5171
step 2700: train loss 2.4828, val loss 2.5025

Men pand, bemary.
Yor 'sour menm sora an hie.
OLond betwe me matand thot suld me th llety ome.
I muco ffepyotssththas l.
TAn.
Momo thapeinthesse ed Pe bene ovetour?
Cassches os cok hedin tie s indo hisor fe f tas :
Whit Cl s;

Tundisthou ldu he n,
Fo mone.

Anthatakes agherchin
NENam s s withoumas Fond thacolld INour id,
Tcks honourd?
TI idurd po veno nond Ce?
Fy
Se thisoou tiund tho nofen e sutan wiporthare whanothavitthers,

O, Bllollke, on s h O, t pan, cr wat hotive wout ir f; u;

Fe ce inee


In [54]:
torch.manual_seed(1337)
B,T,C = 4, 8, 2 # batch size, context length, vocab size
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [62]:
#Let's make the context talk to each other
# Essentially, the value for a specific context will be the average of the values of the context that came before it.
# Ex. If the context is "Matthe" and the next character is "w", then the value for "w" will be the average of the values for "M, a, t, t, h, e, w" 

xbow = torch.zeros((B,T,C))
for batch in range(B):
    for context in range(T):
        xprev = x[batch, :context+1]
        xbow[batch, context] = xprev.mean(dim=0)

# Or alternatively 

wei = torch.tril(torch.ones((T,T)))
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x

torch.allclose(xbow, xbow2) 


# Or alternatively
tril = torch.tril(torch.ones((T,T)))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf')) # Make the 0s -inf
wei = F.softmax(wei, dim=1) # Will equal wei from the prev part (Because softmax will exponentiate the -inf to 0 and 0s to 1 and then normalize)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [45]:
x = torch.zeros((2,3))
display(x)

tensor([[0., 0., 0.],
        [0., 0., 0.]])